In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

Data loading

In [2]:
df=pd.read_csv('laptop.csv')

In [3]:
df.head()

,Unnamed: 0,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,Battery_Life
0,0,HP,HP Chromebook 11A-NA0002MU (2E4N0PA) Laptop (1...,22990,MediaTek Octa-core,MediaTek,Not Expandable,4 GB,DDR4 RAM,2.0 Ghz Processor,LED,11.6,Integrated Graphics,MediaTek,64 GB SSD Storage,No HDD,45,Upto 12 Hrs Battery Life
1,1,Lenovo,Lenovo Ideapad Slim 3 (82KU017KIN) Laptop (15....,36289,AMD Hexa-Core Ryzen 5,AMD,12 GB Expandable,8 GB,DDR4 RAM,4.0 Ghz Processor,LCD,15.6,Radeon,AMD,512 GB SSD Storage,No HDD,65,Upto 11 Hrs Battery Life
2,3,Dell,Dell G15-5520 (D560822WIN9B) Laptop (15.6 Inch...,78500,Intel Core i5 (12th Gen),Intel,32 GB Expandable,16 GB,DDR5 RAM,3.3 Ghz Processor,LCD,15.6,"GeForce RTX 3050 GPU, 4 GB",NVIDIA,512 GB SSD Storage,No HDD,56,Upto 10 Hrs Battery Life
3,4,HP,HP 15s-fy5007TU (91R03PA) Laptop (15.6 Inch | ...,55490,Intel Core i5 (12th Gen),Intel,8 GB Expandable,8 GB,DDR4 RAM,4.2 Ghz Processor,LCD,15.6,Iris Xe,Intel,512 GB SSD Storage,No HDD,no,Upto 7.30 Hrs Battery Life
4,6,Infinix,Infinix Inbook Y2 Plus XL29 Laptop (15.6 Inch ...,21990,Intel Core i3 (11th Gen),Intel,Not Expandable,8 GB LP,LPDDR4X RAM,1.7 Ghz Processor,LCD,15.6,UHD,Intel,512 GB SSD Storage,No HDD,45,Upto 8 Hrs Battery Life


Data cleaning and feature engineering

In [4]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
df.head()

,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,Battery_Life
0,HP,HP Chromebook 11A-NA0002MU (2E4N0PA) Laptop (1...,22990,MediaTek Octa-core,MediaTek,Not Expandable,4 GB,DDR4 RAM,2.0 Ghz Processor,LED,11.6,Integrated Graphics,MediaTek,64 GB SSD Storage,No HDD,45,Upto 12 Hrs Battery Life
1,Lenovo,Lenovo Ideapad Slim 3 (82KU017KIN) Laptop (15....,36289,AMD Hexa-Core Ryzen 5,AMD,12 GB Expandable,8 GB,DDR4 RAM,4.0 Ghz Processor,LCD,15.6,Radeon,AMD,512 GB SSD Storage,No HDD,65,Upto 11 Hrs Battery Life
2,Dell,Dell G15-5520 (D560822WIN9B) Laptop (15.6 Inch...,78500,Intel Core i5 (12th Gen),Intel,32 GB Expandable,16 GB,DDR5 RAM,3.3 Ghz Processor,LCD,15.6,"GeForce RTX 3050 GPU, 4 GB",NVIDIA,512 GB SSD Storage,No HDD,56,Upto 10 Hrs Battery Life
3,HP,HP 15s-fy5007TU (91R03PA) Laptop (15.6 Inch | ...,55490,Intel Core i5 (12th Gen),Intel,8 GB Expandable,8 GB,DDR4 RAM,4.2 Ghz Processor,LCD,15.6,Iris Xe,Intel,512 GB SSD Storage,No HDD,no,Upto 7.30 Hrs Battery Life
4,Infinix,Infinix Inbook Y2 Plus XL29 Laptop (15.6 Inch ...,21990,Intel Core i3 (11th Gen),Intel,Not Expandable,8 GB LP,LPDDR4X RAM,1.7 Ghz Processor,LCD,15.6,UHD,Intel,512 GB SSD Storage,No HDD,45,Upto 8 Hrs Battery Life


In [6]:
df.shape

(3976, 17)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3976 entries, 0 to 3975
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Brand            3976 non-null   object
 1   Name             3976 non-null   object
 2   Price            3976 non-null   int64 
 3   Processor_Name   3976 non-null   object
 4   Processor_Brand  3976 non-null   object
 5   RAM_Expandable   3976 non-null   object
 6   RAM              3976 non-null   object
 7   RAM_TYPE         3976 non-null   object
 8   Ghz              3976 non-null   object
 9   Display_type     3976 non-null   object
 10  Display          3976 non-null   object
 11  GPU              3968 non-null   object
 12  GPU_Brand        3972 non-null   object
 13  SSD              3976 non-null   object
 14  HDD              3976 non-null   object
 15  Adapter          3976 non-null   object
 16  Battery_Life     3558 non-null   object
dtypes: int64(1), object(16)
memory us

In [8]:
df[df['Display']=='OLED Display With Touchscreen']

,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,Battery_Life
774,HP,HP ENVY 15 ep1087TX (54B88PA) Laptop (| Core i...,189400,Intel Core i9 (11th Gen),Intel,Not Expandable,32 GB,DDR4 RAM,4.8 Ghz Processor,LED,OLED Display With Touchscreen,"GeForce RTX 3060 GPU, 6 GB",NVIDIA,1024 GB SSD Storage,No HDD,83,Upto 7.15 Hrs Battery Life


In [9]:
df.drop(774,inplace=True)
# thought of dropping it as ram_expandable,hdd for this row would also be 0 and display=0 doesnt make sense

In [10]:
def adap(num):
    if num.isdigit()==True:
        return int(num)
    else:
        return 0
    
df['Display_size']=df['Display'].astype(np.float32)
df['adapter']=df['Adapter'].apply(adap)
df.drop(columns=['Display','Adapter'],inplace=True)
df['HDD_in_GB']=df['HDD'].str.split().apply(lambda x: int(x[0]) if x[0]!='No' else 0)
df.drop(columns=['HDD'],inplace=True)
df['ghz']=df['Ghz'].str.split().apply(lambda x: float(x[0]))
df.drop(columns=['Ghz'],inplace=True)
df['SSD_in_GB']=df['SSD'].str.split().apply(lambda x: 0 if x[0]=='NO' else int(x[0]))
df.drop(columns=['SSD'],inplace=True)
df['RAM_Expandable_in_GB']=df['RAM_Expandable'].str.split().apply(lambda x: 0 if x[0]=='Not' else int(x[0]))
df.drop(columns='RAM_Expandable',inplace=True)
df['RAM_Size']=df['RAM'].str.split().apply(lambda x: int(x[0]))
df.drop(columns=['RAM'],inplace=True)
df['ram_type']=df['RAM_TYPE'].str.split().apply(lambda x: 'Other' if x[0]=='RAM' else x[0])
df.drop(columns=['RAM_TYPE'],inplace=True)

In [11]:
df.head()

,Brand,Name,Price,Processor_Name,Processor_Brand,Display_type,GPU,GPU_Brand,Battery_Life,Display_size,adapter,HDD_in_GB,ghz,SSD_in_GB,RAM_Expandable_in_GB,RAM_Size,ram_type
0,HP,HP Chromebook 11A-NA0002MU (2E4N0PA) Laptop (1...,22990,MediaTek Octa-core,MediaTek,LED,Integrated Graphics,MediaTek,Upto 12 Hrs Battery Life,11.6,45,0,2.0,64,0,4,DDR4
1,Lenovo,Lenovo Ideapad Slim 3 (82KU017KIN) Laptop (15....,36289,AMD Hexa-Core Ryzen 5,AMD,LCD,Radeon,AMD,Upto 11 Hrs Battery Life,15.6,65,0,4.0,512,12,8,DDR4
2,Dell,Dell G15-5520 (D560822WIN9B) Laptop (15.6 Inch...,78500,Intel Core i5 (12th Gen),Intel,LCD,"GeForce RTX 3050 GPU, 4 GB",NVIDIA,Upto 10 Hrs Battery Life,15.6,56,0,3.3,512,32,16,DDR5
3,HP,HP 15s-fy5007TU (91R03PA) Laptop (15.6 Inch | ...,55490,Intel Core i5 (12th Gen),Intel,LCD,Iris Xe,Intel,Upto 7.30 Hrs Battery Life,15.6,0,0,4.2,512,8,8,DDR4
4,Infinix,Infinix Inbook Y2 Plus XL29 Laptop (15.6 Inch ...,21990,Intel Core i3 (11th Gen),Intel,LCD,UHD,Intel,Upto 8 Hrs Battery Life,15.6,45,0,1.7,512,0,8,LPDDR4X


In [12]:
def brd(text):
    L=['RDP','Nokia','VAIO','Lava','Huawei','Razer','Smartron','Tecno','Colorful','Micromax','Coconics','Realme','Wings','Zebronics','Fujitsu']
    if text in L:
        return 'other brand'
    else:
        return text
    

df['brand']=df['Brand'].apply(brd)
df.drop(columns=['Brand'],inplace=True)
df.info()    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3975 entries, 0 to 3975
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  3975 non-null   object 
 1   Price                 3975 non-null   int64  
 2   Processor_Name        3975 non-null   object 
 3   Processor_Brand       3975 non-null   object 
 4   Display_type          3975 non-null   object 
 5   GPU                   3967 non-null   object 
 6   GPU_Brand             3971 non-null   object 
 7   Battery_Life          3557 non-null   object 
 8   Display_size          3975 non-null   float32
 9   adapter               3975 non-null   int64  
 10  HDD_in_GB             3975 non-null   int64  
 11  ghz                   3975 non-null   float64
 12  SSD_in_GB             3975 non-null   int64  
 13  RAM_Expandable_in_GB  3975 non-null   int64  
 14  RAM_Size              3975 non-null   int64  
 15  ram_type             

In [13]:
df['ghz']=df['ghz'].astype(np.float32)
df['HDD_in_GB']=df['HDD_in_GB'].astype(np.int16)
df['SSD_in_GB']=df['SSD_in_GB'].astype(np.int16)
df['adapter']=df['adapter'].astype(np.int16)
df['RAM_Expandable_in_GB']=df['RAM_Expandable_in_GB'].astype(np.int16)
df['RAM_Size']=df['RAM_Size'].astype(np.int16)

In [14]:
# reason to do like this and not going by processor_brand because i want to separate the intel processor with gen and 
#intel processor without gens
# think whether intel core m3 and m5 to add or not
def proc(text):
    if 'Intel' in text:
        if 'i3' in text or 'i5' in text or 'i7' in text or 'i9' in text:
            return " ".join(text.split()[:3])
        else:
            return 'Other Intel'
    elif 'AMD' in text:
        return " ".join(text.split()[:2])
    elif 'Apple' in text:
        return 'Apple'
    else:
        return 'Other Processor'
    
    
def gen(text):
    if 'Intel' in text:
        if 'i3' in text or 'i5' in text or 'i7' in text or 'i9' in text:
            if text.split()[3][-2:]=='th':
                return int(text.split('th')[0].split('(')[-1])
            elif text.split()[3][-2:]=='st':
                return int(text.split('st')[0].split('(')[-1])
            elif text.split()[3][-2:]=='nd':
                return int(text.split('nd')[0].split('(')[-1])
            elif text.split()[3][-2:]=='rd':
                return int(text.split('rd')[0].split('(')[-1])
            else:
                return 0
                
        else:
            return 0
    else:
        return 0
    
    
def os(text):
    if ('Windows 11' in text.split('|')[3]) or 'Windows 10' in text.split('|')[3] or ('Windows 8.1' in text.split('|')[3]) or ('DOS' in text.split('|')[3]) or ('Linux' in text.split('|')[3]) or ('Ubuntu' in text.split('|')[3]) :
        return text.split('|')[3].strip()
    elif 'macOS' in text.split('|')[3]:
        return 'macOS'
    else:
        return 'Others'
    
    
    
df['processor']=df['Processor_Name'].apply(proc)
df['generation']=df['Processor_Name'].apply(gen)  
df['generation']=df['generation'].astype(np.int16)
df['OpSys']=df['Name'].apply(os)

df.drop(columns=['Name','Processor_Name','Processor_Brand'],inplace=True)

In [15]:
df['GPU_Brand'].fillna(df['GPU_Brand'].value_counts().index[0],inplace=True)
# filled it before train test split because the number of mising values were way too small(only 4) so it wont matter

In [16]:
def brnd(text):
    if 'Intel' in text:
        return 'Intel'
    elif 'NVIDIA' in text or 'Nvidia' in text or 'NIVIDIA' in text:
        return 'NVIDIA'
    elif 'AMD' in text:
        return 'AMD'
    elif 'Apple' in text:
        return 'Apple'
    else:
        return 'Others'
    
    
def gptyp(text):
    L1=df[df['GPU'].notnull()]['GPU'].str.split().apply(lambda x: x[0]).value_counts().index[:8]
    if text in L1:
        return text
    else:
        return 'other gpu'    
    

def adap(text):
    if text.strip()=='65wW Adapter':
        return 65
    elif text.strip()=='45 / 65W Adapter':
        return 45
    else:
        return float((text.split('W'))[0])
    
    
df['adapter']=df['adapter'].astype(np.float32)    
df['gpu_brand']=df['GPU_Brand'].apply(brnd)
df.drop(columns=['GPU_Brand'],inplace=True)
df['gpu_type']=df[df['GPU'].notnull()]['GPU'].str.split().apply(lambda x: x[0]).apply(gptyp)

df2=df[df['Battery_Life'].notnull()]
df3=df2[df2['Battery_Life'].apply(lambda x:'Adapter' in x)][df2[df2['Battery_Life'].apply(lambda x:'Adapter' in x)]['adapter']==0]
df3['adapter']=df3['Battery_Life'].apply(adap)
df2r=df2[df2['Battery_Life'].apply(lambda x:'Adapter' in x)][df2[df2['Battery_Life'].apply(lambda x:'Adapter' in x)]['adapter']==0]
dfcop=df.copy()
dfcop.drop(df2r.index,inplace=True)
data=pd.concat([dfcop,df3])
data.index=list(range(0,3975))
data.drop(columns=['Battery_Life','GPU'],inplace=True)
data['adapter']=data['adapter'].astype(np.float32)

# dropped the battery life column as many of the values were nan values and most other were specifying adapter rather than battery life

In [17]:
data

,Price,Display_type,Display_size,adapter,HDD_in_GB,ghz,SSD_in_GB,RAM_Expandable_in_GB,RAM_Size,ram_type,brand,processor,generation,OpSys,gpu_brand,gpu_type
0,22990,LED,11.6,45.000000,0,2.0,64,0,4,DDR4,HP,Other Processor,0,Others,Others,Integrated
1,36289,LCD,15.6,65.000000,0,4.0,512,12,8,DDR4,Lenovo,AMD Hexa-Core,0,Windows 11,AMD,Radeon
2,78500,LCD,15.6,56.000000,0,3.3,512,32,16,DDR5,Dell,Intel Core i5,12,Windows 11,NVIDIA,GeForce
3,55490,LCD,15.6,0.000000,0,4.2,512,8,8,DDR4,HP,Intel Core i5,12,Windows 11,Intel,Iris
4,21990,LCD,15.6,45.000000,0,1.7,512,0,8,LPDDR4X,Infinix,Intel Core i3,11,Windows 11,Intel,UHD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,102990,LED,14.0,60.700001,0,2.8,1024,0,16,DDR4,Lenovo,Intel Core i7,11,Windows 10,Intel,Iris
3971,22990,LED,14.0,36.700001,0,3.0,256,0,8,DDR4,AVITA,AMD Dual-Core,0,Windows 10,AMD,Radeon
3972,46990,LED,14.0,52.500000,0,1.0,512,0,8,DDR4,Lenovo,Intel Core i5,1,Windows 10,NVIDIA,GeForce
3973,70990,LED,15.6,52.500000,1024,2.3,256,0,8,DDR4,HP,AMD Quad-Core,0,Windows 10,NVIDIA,GeForce


In [18]:
data.head()

,Price,Display_type,Display_size,adapter,HDD_in_GB,ghz,SSD_in_GB,RAM_Expandable_in_GB,RAM_Size,ram_type,brand,processor,generation,OpSys,gpu_brand,gpu_type
0,22990,LED,11.6,45.0,0,2.0,64,0,4,DDR4,HP,Other Processor,0,Others,Others,Integrated
1,36289,LCD,15.6,65.0,0,4.0,512,12,8,DDR4,Lenovo,AMD Hexa-Core,0,Windows 11,AMD,Radeon
2,78500,LCD,15.6,56.0,0,3.3,512,32,16,DDR5,Dell,Intel Core i5,12,Windows 11,NVIDIA,GeForce
3,55490,LCD,15.6,0.0,0,4.2,512,8,8,DDR4,HP,Intel Core i5,12,Windows 11,Intel,Iris
4,21990,LCD,15.6,45.0,0,1.7,512,0,8,LPDDR4X,Infinix,Intel Core i3,11,Windows 11,Intel,UHD


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3975 entries, 0 to 3974
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Price                 3975 non-null   int64  
 1   Display_type          3975 non-null   object 
 2   Display_size          3975 non-null   float32
 3   adapter               3975 non-null   float32
 4   HDD_in_GB             3975 non-null   int16  
 5   ghz                   3975 non-null   float32
 6   SSD_in_GB             3975 non-null   int16  
 7   RAM_Expandable_in_GB  3975 non-null   int16  
 8   RAM_Size              3975 non-null   int16  
 9   ram_type              3975 non-null   object 
 10  brand                 3975 non-null   object 
 11  processor             3975 non-null   object 
 12  generation            3975 non-null   int16  
 13  OpSys                 3975 non-null   object 
 14  gpu_brand             3975 non-null   object 
 15  gpu_type             

Model selection

In [20]:
# defining X and Y

X = data.drop(columns=['Price'])
Y =data['Price']

# performing a train test split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=2)

# filling the missing values after doing train test split
X_train.fillna(X_train['gpu_type'].value_counts().index[0],inplace=True)
X_test.fillna(X_test['gpu_type'].value_counts().index[0],inplace=True)

In [21]:
#linear regression

In [23]:
#taking care of categorical cols using one hot encoding by applying a column transformer

transformer = ColumnTransformer(transformers=[('col_tnf',OneHotEncoder(sparse=False,drop='first'),['Display_type','ram_type','brand','processor','OpSys','gpu_brand','gpu_type'])],
                          remainder='passthrough')

step2 = LinearRegression()

pipe = Pipeline([
    ('step1',transformer),
    ('step2',step2)
])

pipe.fit(X_train,Y_train)
Y_pred = pipe.predict(X_test)

print('R2 score',r2_score(Y_test,Y_pred))
print('MAE',mean_absolute_error(Y_test,Y_pred))
print('MSE',mean_squared_error(Y_test,Y_pred))
print('RMSE',np.sqrt(mean_squared_error(Y_test,Y_pred)))

R2 score 0.7980594771189293
MAE 14533.804878613932
MSE 464150577.91337055
RMSE 21544.15414708525


In [ ]:
#ridge regression

In [ ]:
# using gridsearchCV to find alpha value for ridge regression and applying it

In [33]:
#pipeline for Ridge 
pipe_ridge = Pipeline([
    ('step1', ColumnTransformer(
        transformers=[('col_tnf', OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore'), ['Display_type', 'ram_type', 'brand', 'processor', 'OpSys', 'gpu_brand', 'gpu_type'])],
        remainder='passthrough'
    )),
    ('step2', Ridge())
])

#choosing best value for alpha
param_grid = {
    'step2__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
}

#GridSearchCV to find best alpha
gcv_ridge = GridSearchCV(pipe_ridge, param_grid, cv=5, scoring='r2', verbose=1)
gcv_ridge.fit(X_train, Y_train)
best_alpha = gcv_ridge.best_params_['step2__alpha']
print(f'--- Ridge Regression with GridSearchCV Results ---')
print(f'Best alpha: {best_alpha}\n')

#predictions and performance metrics
best_ridge_model = gcv_ridge.best_estimator_
Y_pred_ridge = best_ridge_model.predict(X_test)

print('R2 score', r2_score(Y_test, Y_pred_ridge))
print('MAE', mean_absolute_error(Y_test, Y_pred_ridge))
print('MSE', mean_squared_error(Y_test, Y_pred_ridge))
print('RMSE', np.sqrt(mean_squared_error(Y_test, Y_pred_ridge)))


Fitting 5 folds for each of 7 candidates, totalling 35 fits


C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Foun

--- Ridge Regression with GridSearchCV Results ---
Best alpha: 0.1

R2 score 0.7981251582324873
MAE 14526.41083672145
MSE 463999613.033311
RMSE 21540.650246297373


In [ ]:
#lasso

In [ ]:
# using gridsearchCV to find alpha value for lasso regression and r2 score

In [39]:
pipe_lasso = Pipeline([
    ('step1', ColumnTransformer(
        transformers=[('col_tnf', OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore'), ['Display_type', 'ram_type', 'brand', 'processor', 'OpSys', 'gpu_brand', 'gpu_type'])],
        remainder='passthrough'
    )),
    ('step2', Lasso(max_iter=10000)) # Increased max_iter for convergence
])

# #choosing best value for alpha
param_grid_lasso = {
    'step2__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
}

#GridSearchCV to find best alpha
gcv_lasso = GridSearchCV(pipe_lasso, param_grid_lasso, cv=5, scoring='r2', verbose=1)
gcv_lasso.fit(X_train, Y_train)
best_alpha_lasso = gcv_lasso.best_params_['step2__alpha']
print(f'--- Lasso Regression with GridSearchCV Results ---')
print(f'Best alpha: {best_alpha_lasso}\n')

# predictions and performance metrics
best_lasso_model = gcv_lasso.best_estimator_
Y_pred_lasso = best_lasso_model.predict(X_test)

print('R2 score', r2_score(Y_test, Y_pred_lasso))
print('MAE', mean_absolute_error(Y_test, Y_pred_lasso))
print('MSE', mean_squared_error(Y_test, Y_pred_lasso))
print('RMSE', np.sqrt(mean_squared_error(Y_test, Y_pred_lasso)))



Fitting 5 folds for each of 7 candidates, totalling 35 fits


C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.352e+11, tolerance: 7.290e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as 

--- Lasso Regression with GridSearchCV Results ---
Best alpha: 10

R2 score 0.7978377765131103
MAE 14510.035418833679
MSE 464660146.0913994
RMSE 21555.977038663765


In [40]:
# taking the price in log as it increases the r2 score

In [41]:
# defining X and Y

X = data.drop(columns=['Price'])
Y =np.log(data['Price'])

# performing a train test split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=2)

# filling the missing values after doing train test split
X_train.fillna(X_train['gpu_type'].value_counts().index[0],inplace=True)
X_test.fillna(X_test['gpu_type'].value_counts().index[0],inplace=True)

In [42]:
#linear regression

In [44]:
#taking care of categorical cols using one hot encoding by applying a column transformer

transformer = ColumnTransformer(transformers=[('col_tnf',OneHotEncoder(sparse=False,drop='first'),['Display_type','ram_type','brand','processor','OpSys','gpu_brand','gpu_type'])],
                          remainder='passthrough')

step2 = LinearRegression()

pipe = Pipeline([
    ('step1',transformer),
    ('step2',step2)
])

pipe.fit(X_train,Y_train)
Y_pred = pipe.predict(X_test)

In [45]:
print('R2 score',r2_score(Y_test,Y_pred))
print('MAE',mean_absolute_error(Y_test,Y_pred))
print('MSE',mean_squared_error(Y_test,Y_pred))
print('RMSE',np.sqrt(mean_squared_error(Y_test,Y_pred)))

R2 score 0.8260371433346492
MAE 0.17619917413345298
MSE 0.05540122274217234
RMSE 0.23537464337131206


In [46]:
# ridge regression

In [50]:
# Ridge Regression with GridSearchCV(doing the ame thing)
pipe_ridge = Pipeline([
    ('step1', ColumnTransformer(
        transformers=[('col_tnf', OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore'), ['Display_type', 'ram_type', 'brand', 'processor', 'OpSys', 'gpu_brand', 'gpu_type'])],
        remainder='passthrough'
    )),
    ('step2', Ridge())
])


param_grid = {
    'step2__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
}


gcv_ridge = GridSearchCV(pipe_ridge, param_grid, cv=5, scoring='r2', verbose=1)
gcv_ridge.fit(X_train, Y_train)
best_alpha = gcv_ridge.best_params_['step2__alpha']
print(f'--- Ridge Regression with GridSearchCV Results (Log Scale) ---')
print(f'Best alpha: {best_alpha}\n')


best_ridge_model = gcv_ridge.best_estimator_
Y_pred_ridge = best_ridge_model.predict(X_test)
print('R2 score', r2_score(Y_test, Y_pred_ridge))
print('MAE', mean_absolute_error(Y_test, Y_pred_ridge))
print('MSE', mean_squared_error(Y_test, Y_pred_ridge))
print('RMSE', np.sqrt(mean_squared_error(Y_test, Y_pred_ridge)))
print('\n')

Fitting 5 folds for each of 7 candidates, totalling 35 fits


C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Foun

--- Ridge Regression with GridSearchCV Results (Log Scale) ---
Best alpha: 0.1

R2 score 0.8262012230723147
MAE 0.1761442603459908
MSE 0.05534896895496662
RMSE 0.23526361587582262




C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
#lasso

In [51]:
# perfmring the same steps
pipe_lasso = Pipeline([
    ('step1', ColumnTransformer(
        transformers=[('col_tnf', OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore'), ['Display_type', 'ram_type', 'brand', 'processor', 'OpSys', 'gpu_brand', 'gpu_type'])],
        remainder='passthrough'
    )),
    ('step2', Lasso(max_iter=10000))
])


param_grid_lasso = {
    'step2__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
}


gcv_lasso = GridSearchCV(pipe_lasso, param_grid_lasso, cv=5, scoring='r2', verbose=1)
gcv_lasso.fit(X_train, Y_train)
best_alpha_lasso = gcv_lasso.best_params_['step2__alpha']
print(f'--- Lasso Regression with GridSearchCV Results (Log Scale) ---')
print(f'Best alpha: {best_alpha_lasso}\n')


best_lasso_model = gcv_lasso.best_estimator_
Y_pred_lasso = best_lasso_model.predict(X_test)
print('R2 score', r2_score(Y_test, Y_pred_lasso))
print('MAE', mean_absolute_error(Y_test, Y_pred_lasso))
print('MSE', mean_squared_error(Y_test, Y_pred_lasso))
print('RMSE', np.sqrt(mean_squared_error(Y_test, Y_pred_lasso)))



Fitting 5 folds for each of 7 candidates, totalling 35 fits


C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\sahil\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Foun

--- Lasso Regression with GridSearchCV Results (Log Scale) ---
Best alpha: 0.0001

R2 score 0.8269357923683673
MAE 0.17563865830407352
MSE 0.05511503374620846
RMSE 0.23476591265813795


Choosing the best model and doing some prediction

In [52]:
# choosing lasso regression with y as log(price)

In [53]:
# defining X and Y

X = data.drop(columns=['Price'])
Y =np.log(data['Price'])

# performing a train test split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=2)

# filling the missing values after doing train test split
X_train.fillna(X_train['gpu_type'].value_counts().index[0],inplace=True)
X_test.fillna(X_test['gpu_type'].value_counts().index[0],inplace=True)

transformer = ColumnTransformer(transformers=[('col_tnf',OneHotEncoder(sparse=False,drop='first'),['Display_type','ram_type','brand','processor','OpSys','gpu_brand','gpu_type'])],
                          remainder='passthrough')

step2 =Lasso(alpha=0.0001)

pipe = Pipeline([
    ('step1',transformer),
    ('step2',step2)
])

pipe.fit(X_train,Y_train)
Y_pred = pipe.predict(X_test)

print('R2 score',r2_score(Y_test,Y_pred))
print('MAE',mean_absolute_error(Y_test,Y_pred))
print('MSE',mean_squared_error(Y_test,Y_pred))
print('RMSE',np.sqrt(mean_squared_error(Y_test,Y_pred)))

R2 score 0.8269357923683673
MAE 0.17563865830407352
MSE 0.05511503374620846
RMSE 0.23476591265813795


In [60]:
# predicting the price 
new_data = pd.DataFrame({
    'Display_type': ['LED'],
    'Display_size': [14],
    'adapter': [65],
    'HDD_in_GB': [0],
    'ghz': [4.5],
    'SSD_in_GB': [512],
    'RAM_Expandable_in_GB': [14],
    'RAM_Size': [16],
    'ram_type': ['DDR5'],
    'brand': ['ASUS'],
    'processor': ['Intel Core i5'],
    'generation': [13],
    'OpSys':['Windows 11'],
    'gpu_brand':['Intel'],
    'gpu_type':['Iris']
})

# Make predictions on the new data
predictions = pipe.predict(new_data)

actualpred=np.exp(predictions)

print(actualpred)

[76430.04106254]
